In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gc

import matplotlib.pyplot as plt
import seaborn as sns

import requests
import xmltodict
import json

from datetime import datetime
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
warnings.filterwarnings(action='ignore')

In [20]:
train = pd.read_parquet('./data/train.parquet')
test = pd.read_parquet('./data/test.parquet')

In [21]:
from sklearn.preprocessing import LabelEncoder

items =  ['day_of_week', 'road_name', 'start_node_name', 'start_turn_restricted', 'end_node_name', 'end_turn_restricted']
train_label = train.copy()
encoder = LabelEncoder()

for i in items:
    encoder = LabelEncoder()
    encoder.fit(train_label[i])
    train_label[i] = encoder.transform(train_label[i])

items =  ['day_of_week', 'road_name', 'start_node_name', 'start_turn_restricted', 'end_node_name', 'end_turn_restricted']
test_label = test.copy()
encoder = LabelEncoder()

for i in items:
    encoder = LabelEncoder()
    encoder.fit(test_label[i])
    test_label[i] = encoder.transform(test_label[i])

In [22]:
y_train = train_label['target'] 
X_train = train_label.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

X_test = test_label.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)



print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 16)
(4701217,)
(291241, 22)


In [23]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, 
                                            stratify=train['base_hour'], random_state=2021)

print(X_tr.shape, y_tr.shape)
print(X_val.shape, y_val.shape)

(3760973, 16) (3760973,)
(940244, 16) (940244,)


In [48]:
import lightgbm as lgb

start_time = time.time()
LR = lgb.LGBMRegressor(random_state=42).fit(X_tr, y_tr,eval_set=[(X_val, y_val)],early_stopping_rounds=10,verbose=10)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


# 최적의 에포크 수 확인
best_num_boost_round = model.best_iteration_
print("Best number of boosting rounds:", best_num_boost_round)
print('Fit time : ', time.time() - start_time)
print(f"tr mae: {mean_absolute_error(y_tr, LR.predict(X_tr))}")
print(f"val mae: {mean_absolute_error(y_val, LR.predict(X_val))}")
print(f"tr r2_score: {r2_score(y_tr, LR.predict(X_tr))}")
print(f"val r2_score: {r2_score(y_val, LR.predict(X_val))}")


[10]	valid_0's l2: 103.58
[20]	valid_0's l2: 72.5546
[30]	valid_0's l2: 61.4745
[40]	valid_0's l2: 55.2651
[50]	valid_0's l2: 50.9295
[60]	valid_0's l2: 47.5758
[70]	valid_0's l2: 45.269
[80]	valid_0's l2: 43.4726
[90]	valid_0's l2: 41.7517
[100]	valid_0's l2: 40.3264
Best number of boosting rounds: 100
Fit time :  15.045395851135254
tr mae: 4.818480947327323
val mae: 4.8187726032646445
tr r2_score: 0.8416475517055051
val r2_score: 0.8415657716291963


In [50]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

xgb_model = xgb.XGBRegressor(random_state=42)

# 조기 종료를 위한 검증 세트 설정
eval_set = [(X_tr, y_tr), (X_val, y_val)]

# 조기 종료 및 학습
xgb_model.fit(X_tr, y_tr,
              eval_set=eval_set,
              eval_metric='mae',
              early_stopping_rounds=10,
              verbose=10)

# 최적의 에포크 수 확인
best_num_boost_round = xgb_model.best_iteration
print("Best number of boosting rounds:", best_num_boost_round)

# 평가
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print("Best number of boosting rounds:", best_num_boost_round)
print('Fit time : ', time.time() - start_time)
print(f"tr mae: {mean_absolute_error(y_tr, LR.predict(X_tr))}")
print(f"val mae: {mean_absolute_error(y_val, LR.predict(X_val))}")
print(f"tr r2_score: {r2_score(y_tr, LR.predict(X_tr))}")
print(f"val r2_score: {r2_score(y_val, LR.predict(X_val))}")

[0]	validation_0-mae:29.62566	validation_1-mae:29.64953
[10]	validation_0-mae:6.00264	validation_1-mae:6.01059
[20]	validation_0-mae:5.27232	validation_1-mae:5.27581
[30]	validation_0-mae:4.92495	validation_1-mae:4.92762
[40]	validation_0-mae:4.62182	validation_1-mae:4.62485
[50]	validation_0-mae:4.46090	validation_1-mae:4.46399
[60]	validation_0-mae:4.33460	validation_1-mae:4.33837
[70]	validation_0-mae:4.20749	validation_1-mae:4.21051
[80]	validation_0-mae:4.11611	validation_1-mae:4.11997
[90]	validation_0-mae:4.03069	validation_1-mae:4.03463
[99]	validation_0-mae:3.98000	validation_1-mae:3.98380
Best number of boosting rounds: 99
Best number of boosting rounds: 99
Fit time :  285.5918638706207
tr mae: 4.818480947327323
val mae: 4.8187726032646445
tr r2_score: 0.8416475517055051
val r2_score: 0.8415657716291963
